In [1]:
import pandas as pd
import yfinance as yf
from bs4 import BeautifulSoup as bs
import requests

# Choosing the stocks that i will work with

Getting all stock symbols of S&P500 from wikipedia

In [2]:
html = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies').content
soup = bs(html, 'html.parser')

In [3]:
# Here is all the <a>SYMBOL</a>
sp500_symbols_html = soup.findAll("a", class_='external text')

In [4]:
# Eventually some wrong strings came with the html, so i put this filter of 4 str only for stocks symbols
sp500_symbols_list = []

for html in sp500_symbols_html:

    symbol = html.text
    
    if len(symbol) <= 4: sp500_symbols_list.append(html.text)

Discovering the top 30 stocks with more volume 

In [5]:
def find_volume(stock_symbol):

    stock = yf.Ticker(stock_symbol)

    volume = stock.info['volume']
    sector = stock.info['sectorKey']

    stock_dict = {stock_symbol : {'Volume':volume, 'Sector':sector}}

    return stock_dict

In [6]:
def all_volumes(symbols_list):

    stocks_dict = dict()

    for symbol in symbols_list:

        try:
            stock_dict = find_volume(symbol)

            stocks_dict.update(stock_dict)

        except:
            error_dict = {symbol: 'Error'}

            stocks_dict.update(error_dict)
    
    return stocks_dict

In [7]:
stocks_dict = all_volumes(sp500_symbols_list)

In [8]:
def remove_errors(dict):
    
    symbols_error = list()

    for key in dict.keys():
        
        value = dict[key]
        
        if value == 'Error':
            symbols_error.append(key)
    
    return symbols_error

In [9]:
remove_error = remove_errors(stocks_dict)
remove_error

['BF.B']

In [10]:
# Poping all the errors stocks

for stock in remove_error:
    stocks_dict.pop(stock)

In [11]:
all_stocks_df = pd.DataFrame(stocks_dict)
all_stocks_df = all_stocks_df.transpose()
all_stocks_df.head()

,Volume,Sector
MMM,1571061,industrials
AOS,354651,industrials
ABT,2550369,healthcare
ABBV,3760786,healthcare
ACN,1235115,technology


In [12]:
# Selecting top 30 with the most volume
all_stocks_df.sort_values('Volume', ascending=False, inplace=True)

top_stocks = list(all_stocks_df[0:30].index)

# Getting all the data

In [1]:
# This function will return the dataframe os one single stock

def find_data_stock(stock_symbol):

    stock = yf.Ticker(stock_symbol)
    df = stock.history(period="10y")

    df['stock name'] = stock_symbol

    return df

In [14]:
# This function will find all data of each stock and return one single dataframe
def find_all_data(stock_symbol_list):

    dfs = []

    for stock_symbol in stock_symbol_list:

        stock_df = find_data_stock(stock_symbol)

        dfs.append(stock_df)
    
    main_df = pd.concat(dfs)
    
    return main_df

In [15]:
main_df = find_all_data(top_stocks)

# Setting up the data

In [16]:
main_df.sample(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,stock name
Date,,,,,,,,
2017-03-22 00:00:00-04:00,14.441760,14.607440,14.296790,14.483180,12288200,0.0,0.0,KMI
2020-03-11 00:00:00-04:00,55.189526,56.794186,54.071670,55.270660,12099100,0.0,0.0,CVS
2021-10-14 00:00:00-04:00,49.376692,50.403427,49.218013,50.310089,26586700,0.0,0.0,INTC
2021-12-08 00:00:00-05:00,176.150497,177.179993,174.750504,176.158005,45254000,0.0,0.0,AMZN
2019-03-08 00:00:00-05:00,24.971364,25.514221,24.926869,25.496422,40860700,0.0,0.0,BAC


In [17]:
main_df['stock name'].unique()

array(['AMD', 'TSLA', 'AAPL', 'F', 'AAL', 'NVDA', 'PFE', 'INTC', 'KEY',
       'BAC', 'CCL', 'AMZN', 'T', 'KMI', 'PYPL', 'BA', 'UBER', 'MSFT',
       'XOM', 'HBAN', 'GOOG', 'BMY', 'META', 'TFC', 'WFC', 'GM', 'CSX',
       'CVS', 'VZ', 'PCG'], dtype=object)

In [18]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 73779 entries, 2014-01-21 00:00:00-05:00 to 2024-01-18 00:00:00-05:00
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          73779 non-null  float64
 1   High          73779 non-null  float64
 2   Low           73779 non-null  float64
 3   Close         73779 non-null  float64
 4   Volume        73779 non-null  int64  
 5   Dividends     73779 non-null  float64
 6   Stock Splits  73779 non-null  float64
 7   stock name    73779 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 5.1+ MB


In [19]:
# Making the column 'date' and reset the index
main_df.reset_index(inplace=True)

Making a dict with the sector of each company

In [20]:
sector_dict = dict()

for stock in top_stocks:

    sector = stocks_dict[stock]['Sector']
    sector_dict[stock] = sector

In [21]:
main_df['Sectors'] = main_df['stock name'].map(sector_dict)

In [22]:
main_df.sample(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,stock name,Sectors
33737,2018-02-21 00:00:00-05:00,12.021339,12.085094,11.865494,11.865494,13464500,0.0,0.0,KMI,energy
61938,2016-12-23 00:00:00-05:00,30.868612,30.903071,30.618768,30.747995,9351200,0.0,0.0,GM,consumer-cyclical
59373,2016-10-14 00:00:00-04:00,36.562941,36.846251,35.874908,36.190594,45418600,0.0,0.0,WFC,financial-services
68529,2023-03-08 00:00:00-05:00,78.354499,78.666629,77.574169,77.847282,6657000,0.0,0.0,CVS,healthcare
34241,2020-02-24 00:00:00-05:00,16.923549,17.165204,16.853393,16.970322,19068300,0.0,0.0,KMI,energy


In [29]:
main_df.to_csv('../data/main_db.csv', index=False)